In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import onnxmltools
import tract

In [2]:
model = Sequential([
    Dense(32, activation='relu', input_dim=100,  name='main_input'),
    Dense(1, activation='sigmoid', name="dense_1"),
])

In [3]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [4]:
data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, epochs=10, batch_size=32)

Epoch 1/10
32/32 [==============================] - 0s 834us/step - loss: 0.7036 - accuracy: 0.4800
Epoch 2/10
32/32 [==============================] - 0s 802us/step - loss: 0.6949 - accuracy: 0.5210
Epoch 3/10
32/32 [==============================] - 0s 853us/step - loss: 0.6934 - accuracy: 0.5070
Epoch 4/10
32/32 [==============================] - 0s 910us/step - loss: 0.6885 - accuracy: 0.5380
Epoch 5/10
32/32 [==============================] - 0s 1ms/step - loss: 0.6819 - accuracy: 0.5600
Epoch 6/10
32/32 [==============================] - 0s 988us/step - loss: 0.6853 - accuracy: 0.5610
Epoch 7/10
32/32 [==============================] - 0s 820us/step - loss: 0.6797 - accuracy: 0.5800
Epoch 8/10
32/32 [==============================] - 0s 913us/step - loss: 0.6763 - accuracy: 0.5730
Epoch 9/10
32/32 [==============================] - 0s 904us/step - loss: 0.6713 - accuracy: 0.5980
Epoch 10/10
32/32 [==============================] - 0s 743us/step - loss: 0.6714 - accuracy: 0.5800


In [5]:
# Save in tf's format, for benchmarking
model.save('my_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_model/assets


In [6]:
# Save the model in ONNX format to pass to tract
onnx_model = onnxmltools.convert_keras(model)
onnxmltools.utils.save_model(onnx_model, 'example.onnx')

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 6
The maximum opset needed by this model is only 9.


In [7]:
input = np.random.random((1,100)).astype(np.float32)

In [8]:
tf_output = model.predict(input)

In [9]:
# Run the model in tract and check output against TensorFlow
tract_output = tract.onnx().model_for_path("example.onnx").into_optimized().into_runnable().run([input])[0].to_numpy()
assert(np.allclose(tf_output, tract_output))

In [10]:
# Save input and reference output for Rust demo
np.savez("io.npz", input=input, output=tf_output)